In [419]:
import tensorflow as tf
#from tensorflow.keras.models import Sequential, Model
import tf_keras as tfk
import tensorflow_probability as tfp
import random
import numpy as np
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

import tf_keras as tfk
tfkl = tfk.layers

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

## Preparation of data

In [420]:
DATA_PATH = "c:\\users\\talibit\\Drug_discovery_data\\alzheimersdata.txt"
data = open(DATA_PATH, 'r').read().splitlines()
new = []
count =0
for molecule in data:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new.append(molecule)

max_seq_len = len(max(new, key=len)) 

l = [['$']+ list(i) for i in new]
#l = [['$']+ [i] for i in new]
data = pad_sequences(l, maxlen=max_seq_len+1, padding="post", value="£",dtype=object) 
print(data.shape)
# so now each smile string is 50 character long element
#data = np.expand_dims(data, -1)
type(data)

(847, 50)


numpy.ndarray

In [421]:
tokenizer = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer.fit_on_texts(data) 
new_data = tokenizer.texts_to_sequences(data) 
new_data = np.array(new_data)
new_data = np.expand_dims(new_data,-1)
new_data.shape

(847, 50, 1)

In [422]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)
print(f'number of words in word_index: {vocab_size}')
print(f'word_index: {word_index}')

number of words in word_index: 37
word_index: {'c': 1, '£': 2, 'C': 3, '(': 4, ')': 5, '1': 6, '2': 7, 'n': 8, 'O': 9, '3': 10, 'N': 11, '=': 12, '$': 13, '-': 14, '[': 15, ']': 16, '4': 17, 'H': 18, 'F': 19, '/': 20, 'o': 21, 'l': 22, 'S': 23, 'B': 24, 'r': 25, '@': 26, 's': 27, '#': 28, '\\': 29, '+': 30, '.': 31, '5': 32, 'I': 33, 'ï': 34, '»': 35, '¿': 36, 'P': 37}


In [423]:
seed = 1
np.random.seed(seed)
full_train, x_test = train_test_split(np.array(new_data), test_size=0.2, random_state=seed)

x_train, x_val = train_test_split(np.array(full_train), test_size=0.1, random_state=seed)

In [424]:
x_train.shape , x_val.shape, x_test.shape

((609, 50, 1), (68, 50, 1), (170, 50, 1))

In [425]:
## Now let's build a simple VAE that encode 50 characters logn smile string into 

latent_size = 10
Prior_distribution = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_size, dtype=tf.float32),
                                        scale_diag=tf.ones(latent_size, dtype=tf.float32))


kl_regularizer= tfpl.KLDivergenceRegularizer(Prior_distribution,
                                                          weight = 1.0,
                                                          use_exact_kl = False, # using analytical formula leads to 
                                                          test_points_fn = lambda q :q.sample(3),
                                                          test_points_reduce_axis = None) #None : all axis are averaged

In [426]:
encoder = tfk.Sequential([
        tfkl.LSTM(128, input_shape=(50,1), return_sequences=True),
        tfkl.Dropout(0.15),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(256, return_sequences=True),
        tfkl.Dropout(0.05),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(512, return_sequences=True),
        tfkl.Dropout(0.25),
        tfkl.Flatten(),
        tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
        tfpl.MultivariateNormalTriL(latent_size,
                                   activity_regularizer = kl_regularizer)


    ])
encoder.summary()

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_245 (LSTM)             (None, 50, 128)           66560     
                                                                 
 dropout_249 (Dropout)       (None, 50, 128)           0         
                                                                 
 max_pooling1d_63 (MaxPooli  (None, 25, 128)           0         
 ng1D)                                                           
                                                                 
 lstm_246 (LSTM)             (None, 25, 256)           394240    
                                                                 
 dropout_250 (Dropout)       (None, 25, 256)           0         
                                                                 
 max_pooling1d_64 (MaxPooli  (None, 12, 256)           0         
 ng1D)                                               

In [427]:

decoder = tfk.Sequential([
    tfkl.Dense(10, activation = 'relu', input_shape =(latent_size,)),
    tfkl.Reshape((latent_size,1)),
    tfkl.LSTM(512, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(256, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(128, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.Flatten(),
    tfkl.Dense(50 * (vocab_size+1)),  # Output 50 * 37 logits
    tfkl.Reshape((50, vocab_size+1)),  # Reshape to (50, 37)
    tfpl.DistributionLambda(lambda t: tfd.Independent(
        tfd.Categorical(logits=t), reinterpreted_batch_ndims=1))

])
decoder.summary()

Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_111 (Dense)           (None, 10)                110       
                                                                 
 reshape_68 (Reshape)        (None, 10, 1)             0         
                                                                 
 lstm_248 (LSTM)             (None, 10, 512)           1052672   
                                                                 
 dropout_252 (Dropout)       (None, 10, 512)           0         
                                                                 
 up_sampling1d_52 (UpSampli  (None, 20, 512)           0         
 ng1D)                                                           
                                                                 
 lstm_249 (LSTM)             (None, 20, 256)           787456    
                                                     

In [428]:
vae = tfk.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs))

In [429]:
optimizer = tfk.optimizers.Adam(learning_rate=0.0005)
optimizer1 = tfk.optimizers.SGD()
vae.compile(optimizer=optimizer, loss=lambda x, pred : -tf.reduce_mean(pred.log_prob(x)))

In [430]:
#vae.fit(x = x_train, y = x_train,validation_data = (x_val,x_val), epochs = 1)
vae.fit(x = x_train, y = np.squeeze(x_train),validation_data = (x_val, np.squeeze(x_val)), epochs = 30)

Epoch 1/30
20/20 [==============================] - 21s 462ms/step - loss: 130.7246 - val_loss: 109.0995
Epoch 2/30
20/20 [==============================] - 6s 284ms/step - loss: 106.6514 - val_loss: 107.7340
Epoch 3/30
20/20 [==============================] - 5s 274ms/step - loss: 105.5858 - val_loss: 106.8224
Epoch 4/30
20/20 [==============================] - 6s 306ms/step - loss: 105.2572 - val_loss: 106.5245
Epoch 5/30
20/20 [==============================] - 5s 272ms/step - loss: 105.0143 - val_loss: 106.4022
Epoch 6/30
20/20 [==============================] - 6s 279ms/step - loss: 105.2777 - val_loss: 106.5018
Epoch 7/30
20/20 [==============================] - 6s 285ms/step - loss: 104.7805 - val_loss: 106.8347
Epoch 8/30
20/20 [==============================] - 5s 265ms/step - loss: 104.9659 - val_loss: 106.3218
Epoch 9/30
20/20 [==============================] - 6s 277ms/step - loss: 104.7745 - val_loss: 106.6059
Epoch 10/30
20/20 [==============================] - 6s 303ms/s

In [431]:
# Evaluate the model on the test set

test_loss = vae.evaluate(x_test)
print("Test loss: {}".format(test_loss))

6/6 [==============================] - 3s 34ms/step - loss: 0.0333
Test loss: 0.0332772321999073


### Reconstruction of some examples

In [432]:
print(data.shape)

random_number = random.randint(0, 846)
print("the initial smile string is:",''.join(data[random_number,:].tolist()))

new_data = tokenizer.texts_to_sequences(data[random_number,:]) 
new_data = np.expand_dims(new_data,0)
new_data.shape
q = encoder(new_data)
q
p = decoder(q.mean())


sequence = p.sample().numpy()
tokenizer.sequences_to_texts(sequence)

(847, 50)
the initial smile string is: $CCc1cc2c(c3cc(OC)c(OC)cc13)C(=O)NC2=O££££££££££££


['$ C C C n O c 1 ) C 2 C n c \\ N c c ( H c F c c c c c c = c c c ( 3 2 c c C £ c c 1 2 N C £ £ £ 1 1']

In [433]:
# Generation
n_samples = 100
samples = Prior_distribution.sample(n_samples)
output_distribution = decoder(samples)
smiles = tokenizer.sequences_to_texts(output_distribution.sample().numpy())
smiles

['$ N C C 1 C c c c 2 - C ( n 2 / O c C 2 B ( 3 ( 4 O ( C c n c ) n 1 c 2 ) 2 c £ 1 ] £ £ £ ] O . £ £',
 '$ C = c 1 N c c n 1 c H C n n 3 c n c ( c C ( C c O c 3 ) c c c C c c \\ c N ) n C ) £ £ 3 £ £ £ £ £',
 '$ C O C = 1 c C ) O C C c c c c c C C n 2 c c ( c 2 n c 2 c c c O 2 n c ) £ c c F £ £ £ £ C £ £ £ £',
 '$ F C c 1 c c 1 c c ) 1 c c ( ) c c c - n F c 4 c I c c c O C c ) ) 2 3 ) ) O c C 4 3 r £ £ c £ £ £',
 '$ C O c c N c ( 1 O C ( n c c n ( ( n = c ) c C 3 3 3 ( c - c c ) c c ( c 2 = c 1 2 O n £ C £ £ £ 1',
 '$ C C C 1 = ( - ) O 1 c n N ( ) c 3 c = C c ) ( c ( 3 ) ) ) C c C c ( ( n l c £ ) c 2 ) £ c = £ C £',
 '$ C C N c c C c 1 [ C c ] C 2 c = n n c c C c ( ) c n n c 4 c c ) c B ( 1 n 2 O c c c C £ £ C £ £ £',
 '$ C = 1 ( c c ( 2 / c ( 1 n C 3 c c c ( c c ) 2 n c n c c c ) c 1 - n C C 4 O 4 £ 1 C c C 2 £ £ O £',
 '$ C O O C c c 1 n ( c C c n c C 3 c c n N N c c c ( C c ) c \\ c 1 4 c C 1 ) £ ] ) ) £ £ 1 £ £ £ £ £',
 '$ C C c c ( c n 1 / 1 n 2 c c n c c 4 ( ) ) c ) n c c ( ) 4 

In [434]:
from rdkit import Chem
def checkSMILES(smiles):
	valid_smiles = []
	for smile in smiles:
		new = ''
		for char in smile:
			if char != '$' and char != '£':	
				new += char

		if Chem.MolFromSmiles(new, sanitize=False) is not None and len(new) > 15:
			valid_smiles.append(new)
			print(new)

	return valid_smiles

In [435]:
checkSMILES(smiles)

I C O C c c C c ( O n c o C C ( c \ c c c C - 3 c n c H F n ) 3 O 2 c O c n 2 o 2 N   2    C 


[16:03:03] SMILES Parse Error: syntax error while parsing: N C C 1 C c c c 2 - C ( n 2 / O c C 2 B ( 3 ( 4 O ( C c n c ) n 1 c 2 ) 2 c  1 ]    ] O .  
[16:03:03] SMILES Parse Error: Failed parsing SMILES ' N C C 1 C c c c 2 - C ( n 2 / O c C 2 B ( 3 ( 4 O ( C c n c ) n 1 c 2 ) 2 c  1 ]    ] O .  ' for input: ' N C C 1 C c c c 2 - C ( n 2 / O c C 2 B ( 3 ( 4 O ( C c n c ) n 1 c 2 ) 2 c  1 ]    ] O .  '
[16:03:03] SMILES Parse Error: syntax error while parsing: C = c 1 N c c n 1 c H C n n 3 c n c ( c C ( C c O c 3 ) c c c C c c \ c N ) n C )   3     
[16:03:03] SMILES Parse Error: Failed parsing SMILES ' C = c 1 N c c n 1 c H C n n 3 c n c ( c C ( C c O c 3 ) c c c C c c \ c N ) n C )   3     ' for input: ' C = c 1 N c c n 1 c H C n n 3 c n c ( c C ( C c O c 3 ) c c c C c c \ c N ) n C )   3     '
[16:03:03] SMILES Parse Error: syntax error while parsing: C O C = 1 c C ) O C C c c c c c C C n 2 c c ( c 2 n c 2 c c c O 2 n c )  c c F     C    
[16:03:03] SMILES Parse Error: Failed parsing

['I C O C c c C c ( O n c o C C ( c \\ c c c C - 3 c n c H F n ) 3 O 2 c O c n 2 o 2 N   2    C ']

### We correctly generated one molecule